# Trying vector codings 

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from Bio.Data import IUPACData
from collections import defaultdict
from itertools import product, combinations
from scipy.spatial.distance import hamming
import scipy.stats as ss
import cvxpy as cp
import sys
import scipy.linalg as sl

sys.path.append("../") 
import doe_utils

# TEST MAKE AMINO ACID WORLD SMALL 

AMINO_ACIDS = doe_utils.AMINO_ACIDS

IndentationError: expected an indented block after 'for' statement on line 512 (doe_utils.py, line 519)

In [ ]:
length_of_protein = 100
num_mutated = 4
sold_mat_df, parent, mut_dict = doe_utils.create_synthetic_SOLD_matrix(num_mutated, length_of_protein, mut_probs = [0.1, 0.1], parent_prob=0.80) #6 postions mutated, pprtein length is 100 
sold_mat_df.head() 

In [ ]:
plt.figure(figsize = (18,5))
plt.imshow(sold_mat_df, aspect = 'auto', cmap = 'grey_r') 
_  = plt.xticks(range(length_of_protein), rotation = 90)
_  = plt.yticks(range(len(AMINO_ACIDS)), AMINO_ACIDS)


In [ ]:
S = doe_utils.SOLD(sold_mat_df)
seqs, mutations = S.generate_sequences(10000)
probs = S.compute_prob_n_mutations(num_mutated+1)

In [ ]:
S.mutation_probs_variable_region_indexed

# Change this to make variable length SOLD mat 

In [ ]:
mutated_dict = S.mutation_probs_variable_region_indexed.copy() 
mutated_dict[0] = {'A': 0.1, 'G': 0.1, 'N': 0.7, 'P': 0.1}
mutated_dict[3] = {'C': 0.1, 'D': 0.6, 'E': 0.1, 'W': 0.1, 'K':0.1}
mutated_dict

# Now you have variable length mat to create encoder 

In [ ]:

def create_code(bases): 
    """
    Create a code of length L -- one hot 
    """
    L = len(bases) 
    binary = np.zeros(L) 
    mapper = dict()
    for i,a in enumerate(bases): 
        loc_binary = np.copy(binary)
        loc_binary[i] = 1 
        mapper[a] = loc_binary 
    return mapper 



class Encoding_basics: 
    """
    Base class used by both ecnoders and in silico model 
    """
    def __init__(self, mutation_probs_variable_region_dict): 
        """
        Args: 
            mutated region length: the number of positions mutated 
            mutation_probs_variable_region_dict: the standard way to encode the mutations --- see notes and guide 
        """
        self.mutated_region_length = len(mutation_probs_variable_region_dict)
        self.mutation_probs_variable_region_dict = mutation_probs_variable_region_dict

        # All mutations and encodings are relative to parent 
        # get product aminos and pos products 
        self.amino_product = [''.join(x) for x in product(AMINO_ACIDS, AMINO_ACIDS)]
        self.pos_product = [np.asarray(x) for x in combinations(np.arange(self.mutated_region_length), 2)]


        # now, for every position I need to encode the bases in a minimal code 
        # Create a code dict for every position 
        parent = ['']*len(mutation_probs_variable_region_dict) 
        self.independent_code_mapper = {} 
        single_address = defaultdict(list) 
        self.positions = [] 
        for k, v in mutation_probs_variable_region_dict.items(): 
            code_length = len(v) - 1 
            parent_base = max(v, key=v.get)
            bases = [a for a in v.keys() if a!= parent_base] 
            mapper = create_code(bases) 
            mapper.update({parent_base: np.zeros(code_length)}) 
            parent[k] = max(v, key=v.get)
            self.independent_code_mapper[k] = mapper 
            self.positions.append(k) 
        self.parent = ''.join(parent)
        self.positions = np.sort(self.positions)
        # Now I have all the independent codes 
        #Lets go through all the positions are create the pairwise codes  

        code_size = defaultdict(int) 
        pair_bases = defaultdict(list) 
        for i, j in self.pos_product: 
            for a, b in self.amino_product: 
                if (a in self.independent_code_mapper[i]) and (b in self.independent_code_mapper[j]): 
                    code_size[str(i) + ':' + str(j)] += 1 
                    pair_bases[str(i) + ':' + str(j)].append(a + b)

        self.pairwise_code_mapper = {}
        for i, j in self.pos_product: 
            parent_pair = self.parent[i] + self.parent[j] 
            temp = pair_bases[str(i) + ':' + str(j)]
            code_length = len(temp) - 1 
            bases = [a for a in temp if a!= parent_pair] 
            mapper = create_code(bases) 
            mapper.update({parent_pair: np.zeros(code_length)}) 
            self.pairwise_code_mapper[str(i) + ':' + str(j)] = mapper 

        # Now I need to get the order consistent and create "matrix" --- list of list really because the code sizes are diffferent 
        self.feature_names_independent = []
        # again get the order consistent from AMINO order 
        self.independent_codes_list = [] 
        for s in self.positions: 
            for a in AMINO_ACIDS: 
                if a in self.independent_code_mapper[s]: 
                    self.feature_names_independent.append(str(s) + '-' + a) 
                    self.independent_codes_list.append(self.independent_code_mapper[s][a])
        
        
        self.feature_names_pairwise = [] 
        self.pairwise_codes_list = []            
        for i, j in self.pos_product: 
            for a, b in self.amino_product: 
                if (a in self.independent_code_mapper[i]) and (b in self.independent_code_mapper[j]): 
                    name = str(i) + '-' + str(a) + ':' + str(j) + '-' + str(b) 
                    self.feature_names_pairwise.append(name) 
                    self.pairwise_codes_list.append(self.pairwise_code_mapper[str(i) + ':' + str(j)][str(a) + str(b)])
        
        self.code_length_independent = np.sum([len(code) for code in self.independent_codes_list]) 
        self.code_length_pairwise = np.sum([len(code) for code in self.pairwise_codes_list]) 

class Sequence_encoder_simplex(Encoding_basics): 
    """
    Encodes sequence in simplex encodings --- this one doesn't try to create a general encoder but a SOLD library specifci encoder, which makes more sense anyway 
    Why encode variations / mutations you will never encounter? 
    """
    def __init__(self, mutation_probs_variable_region_dict): 
        """
        Args:
            mutated_region_length: the length of ONLY the variable region (mutated region) of the protein, NOT the whole protein length! 
        """
        super().__init__(mutation_probs_variable_region_dict) 
        
    def encode_seqs(self, protein_seqs): 
        """
        Args: 
            protein_seqs: this is only the variable region 
        """
        independent_codes = [] 
        pairwise_codes = [] 
        for seq in protein_seqs:
            array_of_seq = np.asarray(list(seq))
            assert len(seq) == self.mutated_region_length, "length mismatch of protein seq and attributes"
            
            local_code_I = np.zeros(self.shape_independent_weights)
            for i in range(self.mutated_region_length): 
                feature = str(i) + '-' + array_of_seq[i] 
                index = list(self.feature_names_independent).index(feature) 
                local_code_I[index, :] = self.independent_codes[index, :] 
            independent_codes.append(local_code_I)
            
            local_code_J = np.zeros(self.shape_pairwise_weights)
            for j, pos in enumerate(self.pos_product): 
                # need to find the index of amino_acid pairs 
                acid_pairs = array_of_seq[pos]
                feature = str(pos[0]) + '-' + acid_pairs[0] + ':' + str(pos[1]) + '-' + acid_pairs[1]
                index = list(self.feature_names_pairwise).index(feature)
                local_code_J[index, :] = self.pairwise_codes[index, :]
            pairwise_codes.append(local_code_J) 
            
        independent_codes = np.asarray(independent_codes)
        pairwise_codes = np.asarray(pairwise_codes)

        return independent_codes, pairwise_codes


#######################################################################################################

class Create_in_silico_model(Encoding_basics): 
    """
    Create an in silico model for simulation with independent and pairwise contributions 
    """
    def __init__(self, mutation_probs_variable_region_dict, independent_params = None, pairwise_params = None, baseline = 10): 
        """
        Args:
            mutation_probs ..  : pass the dict of mutation probs (this is generated by SOLD matrix class, attribute dict is called mutation_probs_variable_region_indexed) 
            MUST BE INDEXED BY THE POSITION OF THE MUTATED REGION, not the protein position... this is to make sure we can deal with different length proteins 
                example:
                {0: {'D': 0.05, 'K': 0.85, 'M': 0.05, 'Y': 0.05},
                 1: {'C': 0.05, 'G': 0.05, 'I': 0.05, 'P': 0.85},
                 2: {'F': 0.05, 'N': 0.05, 'R': 0.85, 'Y': 0.05},
                 3: {'G': 0.05, 'I': 0.05, 'L': 0.85, 'Q': 0.05},
                 4: {'A': 0.05, 'E': 0.05, 'R': 0.05, 'W': 0.85},
                 5: {'A': 0.05, 'D': 0.05, 'I': 0.85, 'K': 0.05}}
                where the keys are the index of the mutated regions and the values are dicts of probs of each amino acid
            idependent_params: {'rho':[0.2, 0.2], 'sparse_pdf_names': ['norm', 'norm'], 'noise_sigma' : 0.01, 'sparse_params': [{'loc': 1, 'scale': 0.2}, {'loc': -1, 'scale': 0.2}]} 
            pairwise_params: {'rho':[0.1, 0.1], 'sparse_pdf_names': ['norm', 'norm'], 'noise_sigma' : 0.01, 'sparse_params': [{'loc': 0.75, 'scale': 0.2}, {'loc': -0.75, 'scale': 0.2}]} 
            
        """
        # independent params pdf default --- 
        I_defaults = {'rho':[0.2, 0.2], 'sparse_pdf_names': ['norm', 'norm'], 'noise_sigma' : 0.01, 'sparse_params': [{'loc': 1, 'scale': 0.2}, {'loc': -1, 'scale': 0.2}]} 
        #pairwise params pdf default ---
        P_defaults = {'rho':[0.2, 0.2], 'sparse_pdf_names': ['norm', 'norm'], 'noise_sigma' : 0.01, 'sparse_params': [{'loc': 0.75, 'scale': 0.2}, {'loc': -0.75, 'scale': 0.2}]} 

        super().__init__(mutation_probs_variable_region_dict) 

        if independent_params is not None: 
            I_defaults.update(independent_params) 
        self.independent_params = I_defaults.copy()  

        if pairwise_params is not None: 
            P_defaults.update(pairwise_params) 
        # see the create mixture function
        self.pairwise_params = P_defaults.copy() 
        
        # Now I need to assign weights to individual and pairwise contributions 
        # first fill in the weights for the independent codes
        self.Prob_I = Create_mixture(**self.independent_params)
        samples_I, _ = self.Prob_I.samples(np.prod(self.shape_independent_weights))
        self.independent_weights = samples_I.reshape(self.shape_independent_weights)         
        self.Prob_P = Create_mixture(**self.pairwise_params)
        samples_P, _ = self.Prob_P.samples(np.prod(self.shape_pairwise_weights))
        self.pairwise_weights = samples_P.reshape(self.shape_pairwise_weights)              
        self.baseline = baseline    
    
    def model(self, independent_codes, pairwise_codes): 
        """
        Args: 
            independent_codes: the result of encoding my sequence encoder to independent codes --- these are tensors--- N seqs times A amino acids time L positions (shape_independet_weights) etc. 
            pairwise_codes: similar 
            masked: ignore the weights of independent and pairwise positions that are not variable! 
        """
        ans1 = np.einsum('ijk, jk -> i', independent_codes, self.independent_weights) 
        ans2 = np.einsum('ijk, jk -> i', pairwise_codes, self.pairwise_weights)
        return self.baseline + ans1 + ans2 
        
    def plot_weights(self): 
        """
        Plotting functions 
        """
        # weights 
        I_samples = np.ravel(self.independent_weights) 
        GRID_SIZE = 1000
        x = np.linspace(np.min(I_samples), np.max(I_samples), GRID_SIZE)
        plt.figure() 
        plt.plot(x, self.Prob_I.pdf()(x))
        _ = plt.hist(I_samples, density=True, bins = 50)        
        plt.title('Distribution of independent weights') 

        P_samples = np.ravel(self.pairwise_weights) 
        x = np.linspace(np.min(P_samples), np.max(P_samples), GRID_SIZE)
        plt.figure() 
        plt.plot(x, self.Prob_P.pdf()(x))
        _ = plt.hist(P_samples, density=True, bins = 50)        
        plt.title('Distribution of pairwise weights') 
        plt.figure() 
        plt.imshow(self.independent_weights, vmin = -1, vmax = 1, cmap = 'RdBu', interpolation = 'None') 
        _  = plt.yticks(range(len(self.feature_names_independent)), self.feature_names_independent)
        _  = plt.xticks(range(self.shape_independent_weights[1]))
        plt.title("Independent weights") 
        plt.colorbar() 
        plt.figure(figsize = (5, 70))
        plt.imshow(self.pairwise_weights, vmin = -1, vmax = 1, cmap = 'RdBu', aspect = 'auto', interpolation = 'None') 
        _  = plt.yticks(range(len(self.feature_names_pairwise)), self.feature_names_pairwise)
        _  = plt.xticks(range(self.shape_pairwise_weights[1]))
        plt.title("Pairwise weights") 
        #plt.colorbar



In [ ]:
S = Encoding_basics(mutated_dict)

In [ ]:
S.feature_names_pairwise

In [ ]:
S.code_size

In [ ]:
S.independent_code_mapper[0]['A']

In [ ]:
S.independent_code_mapper[1]['C']

In [ ]:
S.parent


In [ ]:
np.kron([1, 2, 3], [1, 2, 3, 4, 5])